In [ ]:
#|default_exp utils 

Based on Piotr Czapla:  
https://github.com/PiotrCzapla/course22p2/blob/main/nbs/12_utils.ipynb

In [ ]:
#| export

from miniai.learner import Callback
from miniai.learner import MetricsCB, master_bar, progress_bar, to_cpu

import numpy as np
import fastcore.all as fc

In [ ]:
#| export
import time

# Simplified from Piotr

def _format_stats(stats):
    return f"{stats.mean():.2f} ± {stats.std():.2f}"

class TimeItSimpleCB(Callback):
    def __init__(self): self.reset()
    def tick(self): return time.time()
         
    def reset(self):
        self.start = self.tick()
        self.batches = {True:[], False:[]}
        self.epochs = {True:[], False:[]}
        
    def before_fit(self, learn): self.reset()
        
    def before_batch(self, learn):
        self.batches[learn.training].append(self.tick())
    
    def after_batch(self, learn):
        self.batches[learn.training][-1] = self.tick() - self.batches[learn.training][-1]
    
    def before_epoch(self, learn):
        self.epochs[learn.training].append(self.tick())
    
    def after_epoch(self, learn):
        self.epochs[learn.training][-1] = self.tick() - self.epochs[learn.training][-1]
    
    def after_fit(self, learn):
        self.total = self.tick() - self.start
        self.batches = {k:np.array(v) for k,v in self.batches.items()}
        self.epochs = {k:np.array(v) for k,v in self.epochs.items()}
        self.print_stats()
    
    def print_stats(self): 
        print(f"Fit {len(self.epochs[True])} in: {self.total:.2f}s, {_format_stats(self.epochs[True])}s per epoch, {_format_stats(self.batches[True])}s per batch")

In [ ]:
#| export

# Same as Piotr

# if spawn is not possible, we can cache the entire dataset in memory after transformations
def _with_features(ds):
    setattr((l:=fc.L(ds)), 'features', ds.features)
    return l 

class CachedDS(dict): 
    def __repr__(self): return "{ "+", ".join([f'{k}: (#{len(v)})' for k,v in self.items()])+" }"
    def __str__(self): return repr(self)

def cache_dataset_as_dict(dd): return CachedDS({dsn: _with_features(ds) for dsn,ds in dd.items()})

In [ ]:
#| export

# Same as Piotr

class LazyProgressCB(Callback):
    order = MetricsCB.order+1
    def __init__(self, plot=False): 
        self.plot = plot
        self.count = 0
    def before_fit(self, learn):
        learn.epochs = self.mbar = master_bar(learn.epochs)
        self.first = True
        if hasattr(learn, 'metrics'): learn.metrics._log = self._log
        self.dev_losses = []
        self.losses = []

    def _log(self, d):
        if self.first:
            self.mbar.write(list(d), table=True)
            self.first = False
        self.mbar.write(list(map(str, d.values())), table=True)
        if self.plot:
            for l in self.dev_losses: self.losses.append(to_cpu(l))
            self.dev_losses=[]
            #print(self.losses[:10])
            self.mbar.update_graph([[fc.L.range(self.losses), self.losses]])
        
    def before_epoch(self, learn): learn.dl = progress_bar(learn.dl, leave=False, parent=self.mbar)
    def after_batch(self, learn):
        #learn.dl.comment = f'{learn.loss:.3f}' # not sure how it is being used
        if self.plot and hasattr(learn, 'metrics') and learn.training:
            self.dev_losses.append(learn.loss.detach())

In [ ]:
import nbdev; nbdev.nbdev_export()

/home/fmussari/mambaforge/envs/fastaiv3/lib/python3.9/site-packages/nbdev/export.py:54: UserWarning: Notebook '/mnt/m/2023-02_fastai-course22p2/course22p2/nbs/challenge/2023-02. Curricular MixIn.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
